In [55]:
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from imutils import face_utils
font = cv2.FONT_HERSHEY_SIMPLEX

In [56]:
import os

def get_files(path):
    return os.listdir(path)

cascPath = "/Users/abdulrehman/opt/anaconda3/envs/Face-Detection/lib/python3.6/site-packages/cv2/data/haarcascade_frontalface_default.xml"

def return_bbx(image):
    faceCascade = cv2.CascadeClassifier(cascPath)
    faces = faceCascade.detectMultiScale(image, scaleFactor=1.1, minNeighbors=5, flags=cv2.CASCADE_SCALE_IMAGE)
    return faces

In [57]:
Dataset_path = '/Users/abdulrehman/Desktop/SML Project/FacesInTheWild/'

Celebs = pd.read_csv(Dataset_path+'lfw_allnames.csv')
Celebs = Celebs[Celebs['images']>50]

In [58]:
X = []
Y = []

for _, [name,__] in Celebs.iterrows():
    celeb_path = Dataset_path+'lfw-deepfunneled/'+name+'/'
    
    images_paths = get_files(celeb_path)
    for image_path in images_paths:
        image = cv2.imread(celeb_path+image_path,1)
        faces = return_bbx(image)
        if len(faces) == 1:
            (x,y,w,h) = faces[0]
            cropped = image[x:x+w, y:y+h]
            dim = (224, 224)
            resized = cv2.resize(cropped, dim, interpolation = cv2.INTER_AREA)
            image = np.array(resized).astype("float32")
            X.append(image)
            Y.append(name)
X_data = np.array(X)
Y_data = np.array(Y)

In [59]:
X_data = np.array(X)
Y_data = np.array(Y)

print(X_data.shape)
print(Y_data.shape)

(1431, 224, 224, 3)
(1431,)


In [60]:
from collections import Counter

counter = Counter(Y_data)
print(counter)

Counter({'George_W_Bush': 480, 'Colin_Powell': 213, 'Tony_Blair': 138, 'Donald_Rumsfeld': 109, 'Gerhard_Schroeder': 106, 'Ariel_Sharon': 70, 'Hugo_Chavez': 67, 'Jacques_Chirac': 51, 'Jean_Chretien': 51, 'Junichiro_Koizumi': 51, 'Serena_Williams': 48, 'John_Ashcroft': 47})


In [61]:
from imblearn.under_sampling import NearMiss

X_data_reshaped = X_data.reshape(X_data.shape[0], X_data.shape[1] * X_data.shape[2] * X_data.shape[3])
undersample = NearMiss(version=1)
X_resampled, Y_resampled = undersample.fit_resample(X_data_reshaped,Y_data)
X_data = X_resampled.reshape(X_resampled.shape[0], X_data.shape[1], X_data.shape[2], X_data.shape[3])
Y_data = Y_resampled
counter = Counter(Y_data)
print(counter)

del X_data_reshaped
del undersample
del X_resampled
del Y_resampled
del counter

Counter({'Ariel_Sharon': 47, 'Colin_Powell': 47, 'Donald_Rumsfeld': 47, 'George_W_Bush': 47, 'Gerhard_Schroeder': 47, 'Hugo_Chavez': 47, 'Jacques_Chirac': 47, 'Jean_Chretien': 47, 'John_Ashcroft': 47, 'Junichiro_Koizumi': 47, 'Serena_Williams': 47, 'Tony_Blair': 47})


In [62]:
print(X_data.shape)
print(Y_data.shape)

(564, 224, 224, 3)
(564,)


In [63]:
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder

encoder = OneHotEncoder(sparse=False)
# transform data
onehot_Y = encoder.fit_transform(Y_data.reshape(-1,1))
onehot_Y

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [64]:
scaler = MinMaxScaler(feature_range=(0, 1))
X_reshaped = X_data.reshape(X_data.shape[0],X_data.shape[1]*X_data.shape[2]*X_data.shape[3])
X_reshaped = scaler.fit_transform(X_reshaped)
X_data = X_reshaped.reshape(X_data.shape[0],X_data.shape[1],X_data.shape[2],X_data.shape[3])
X_data.shape

(564, 224, 224, 3)

In [65]:
import keras
import tensorflow as tf
from keras.applications import VGG16

model = VGG16(weights = 'imagenet',include_top = True, input_shape = (224,224,3))

i=0
for layer in model.layers:
    if i < 22:
        layer.trainable = False
    i+=1

out = [layer.output for layer in model.layers][21]
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [47]:
model.__dict__

{'_self_setattr_tracking': True,
 '_trainable': True,
 '_stateful': False,
 'built': True,
 '_build_input_shape': TensorShape([None, 224, 224, 3]),
 '_saved_model_inputs_spec': TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_9'),
 '_input_spec': None,
 '_supports_masking': True,
 '_name': 'vgg16',
 '_activity_regularizer': None,
 '_trainable_weights': [],
 '_non_trainable_weights': [],
 '_updates': [],
 '_thread_local': <_thread._local at 0x7f9ad4d3a0f8>,
 '_callable_losses': [],
 '_losses': [],
 '_metrics': [],
 '_metrics_lock': <unlocked _thread.lock object at 0x7f9acb6be760>,
 '_dtype_policy': <Policy "float32", loss_scale=None>,
 '_dtype_defaulted_to_floatx': True,
 '_compute_dtype_object': tf.float32,
 '_autocast': False,
 '_layers': [<tensorflow.python.keras.engine.input_layer.InputLayer at 0x7f9ad4d24be0>,
 '_inbound_nodes': [],
 '_outbound_nodes': [],
 '_expects_training_arg': True,
 '_default_training_arg': None,
 '_expects_mask_arg': True,
 '_dynamic': Fal

In [48]:
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D
from keras.models import Model
from keras.optimizers import Adam
from keras.layers.advanced_activations import LeakyReLU

hidden_layer = out
# hidden_layer = Conv2D(256, (5, 5), activation='relu')(hidden_layer)
# hidden_layer = Flatten()(hidden_layer)
# hidden_layer = Dense(4096, activation='linear')(hidden_layer)
# hidden_layer = LeakyReLU(alpha=0.1)(hidden_layer)
# hidden_layer = Dropout(0.5)(hidden_layer)
hidden_layer = Dense(1028, activation='linear')(hidden_layer)
hidden_layer = LeakyReLU(alpha=0.1)(hidden_layer)
hidden_layer = Dropout(0.5)(hidden_layer)
hidden_layer = Dense(128, activation='linear')(hidden_layer)
hidden_layer = LeakyReLU(alpha=0.1)(hidden_layer)
hidden_layer = Dropout(0.5)(hidden_layer)
out = Dense(12, activation='softmax')(hidden_layer)

In [49]:
model = keras.Model(inputs = model.input, outputs = out)

In [50]:
model.summary()

Model: "functional_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)     

In [51]:
model.compile(optimizer=Adam(lr=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

In [52]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_data, onehot_Y, train_size=0.9, random_state = 0)

In [54]:
epochs = 25
batch_size = 16

model.fit(X_train,y_train,validation_data = (X_test,y_test), verbose = 1,
                       epochs = epochs, batch_size = batch_size)

Epoch 1/25
32/32 [==============================] - 61s 2s/step - loss: 67.4495 - accuracy: 0.1105 - val_loss: 9.2721 - val_accuracy: 0.1579
Epoch 2/25
32/32 [==============================] - 66s 2s/step - loss: 20.3887 - accuracy: 0.0907 - val_loss: 6.8898 - val_accuracy: 0.0702
Epoch 3/25
32/32 [==============================] - 65s 2s/step - loss: 14.2034 - accuracy: 0.0848 - val_loss: 4.7809 - val_accuracy: 0.0877
Epoch 4/25
32/32 [==============================] - 69s 2s/step - loss: 9.8365 - accuracy: 0.1026 - val_loss: 4.0897 - val_accuracy: 0.1053
Epoch 5/25
32/32 [==============================] - 70s 2s/step - loss: 7.9843 - accuracy: 0.0868 - val_loss: 3.1761 - val_accuracy: 0.0702
Epoch 6/25
32/32 [==============================] - 70s 2s/step - loss: 5.8418 - accuracy: 0.0828 - val_loss: 3.1675 - val_accuracy: 0.1228
Epoch 7/25
32/32 [==============================] - 73s 2s/step - loss: 5.2533 - accuracy: 0.1065 - val_loss: 2.8014 - val_accuracy: 0.0702
Epoch 8/25
32/32 

KeyboardInterrupt: 